In [0]:
%pip install spacy
%pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0.tar.gz

Python interpreter will be restarted.
  Using cached spacy-3.8.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.4 MB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl (17 kB)
  Using cached typer-0.15.2-py3-none-any.whl (45 kB)
  Using cached murmurhash-1.0.12-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (124 kB)
  Using cached cymem-2.0.11-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (208 kB)
  Using cached preshed-3.0.9-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (157 kB)
  Using cached weasel-0.4.1-py3-none-any.whl (50 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached srsly-2.5.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
  Using cached langcodes-3.5.0-py3-none-any.whl (182 kB)
  Using cached s

In [0]:
import spacy

In [0]:
nlp = spacy.load("en_core_web_sm")

In [0]:
stop_words = spacy.lang.en.stop_words.STOP_WORDS 

In [0]:
stop_words

Out[4]: {"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from'

In [0]:
for label in nlp.get_pipe("ner").labels:
    print(f"{label}: {spacy.explain(label)}")

CARDINAL: Numerals that do not fall under another type
DATE: Absolute or relative dates or periods
EVENT: Named hurricanes, battles, wars, sports events, etc.
FAC: Buildings, airports, highways, bridges, etc.
GPE: Countries, cities, states
LANGUAGE: Any named language
LAW: Named documents made into laws.
LOC: Non-GPE locations, mountain ranges, bodies of water
MONEY: Monetary values, including unit
NORP: Nationalities or religious or political groups
ORDINAL: "first", "second", etc.
ORG: Companies, agencies, institutions, etc.
PERCENT: Percentage, including "%"
PERSON: People, including fictional
PRODUCT: Objects, vehicles, foods, etc. (not services)
QUANTITY: Measurements, as of weight or distance
TIME: Times smaller than a day
WORK_OF_ART: Titles of books, songs, etc.


In [0]:
nlp.get_pipe("ner").labels

Out[6]: ('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

In [0]:
# TEXT_PATH = "dbfs:/FileStore/tables/large_text_file.txt"
TEXT_PATH = "dbfs:/FileStore/tables/rj.txt"

In [0]:
text = sc.textFile(TEXT_PATH)

In [0]:
text.take(10)

Out[9]: ['The Project Gutenberg eBook of Romeo and Juliet',
 '    ',
 'This ebook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this ebook or online',
 'at www.gutenberg.org. If you are not located in the United States,',
 'you will have to check the laws of the country where you are located',
 'before using this eBook.',
 '']

# Removing Stopwords and extracting labels of words

In [0]:
def process_text(text):
    doc = nlp(text)
    filtered_tokens = [token for token in doc if not token.is_stop]
    # Use map to extract entity tuples without an explicit for loop
    entities = list(map(lambda ent: (ent.text, ent.label_), doc.ents))
    return entities

In [0]:
labelled_entities = text.flatMap(process_text)

In [0]:
type(labelled_entities)

Out[12]: pyspark.rdd.PipelinedRDD

In [0]:
labelled_entities.take(10)

Out[13]: [('Project Gutenberg', 'PERSON'),
 ('Romeo', 'GPE'),
 ('Juliet', 'NORP'),
 ('the United States', 'GPE'),
 ('the Project Gutenberg License', 'ORG'),
 ('the United States', 'GPE'),
 ('eBook', 'ORG'),
 ('Juliet', 'NORP'),
 ('William Shakespeare', 'PERSON'),
 ('November 1, 1998', 'DATE')]

In [0]:
labelled_entities.values().distinct().take(20)

Out[14]: ['PERSON',
 'DATE',
 'CARDINAL',
 'TIME',
 'WORK_OF_ART',
 'ORDINAL',
 'GPE',
 'NORP',
 'ORG',
 'LANGUAGE',
 'PRODUCT',
 'LOC',
 'EVENT',
 'FAC',
 'MONEY',
 'LAW',
 'QUANTITY',
 'PERCENT']

In [0]:
ALLOWED_LABELS = (
 'FAC',
 'GPE',
 'LANGUAGE',
 'LOC',
 'ORG',
 'PERSON',
 'PRODUCT',
 'WORK_OF_ART'
)

## Filtering the named entities by allowed labels.

In [0]:
named_entities = labelled_entities.filter(lambda entity: entity[1] in ALLOWED_LABELS)

In [0]:
named_entities.take(10)

Out[17]: [('Project Gutenberg', 'PERSON'),
 ('Romeo', 'GPE'),
 ('the United States', 'GPE'),
 ('the Project Gutenberg License', 'ORG'),
 ('the United States', 'GPE'),
 ('eBook', 'ORG'),
 ('William Shakespeare', 'PERSON'),
 ('eBook', 'ORG'),
 ('English', 'LANGUAGE'),
 ('Shakespeare Team', 'PERSON')]

# Map Reduce

## Map Phase

In [0]:
named_entities_map = named_entities.map(lambda entity: [entity[0], 1])

In [0]:
named_entities_map.take(10)

Out[19]: [['Project Gutenberg', 1],
 ['Romeo', 1],
 ['the United States', 1],
 ['the Project Gutenberg License', 1],
 ['the United States', 1],
 ['eBook', 1],
 ['William Shakespeare', 1],
 ['eBook', 1],
 ['English', 1],
 ['Shakespeare Team', 1]]

## Reduce Phase

In [0]:
named_entities_reduced = named_entities_map.reduceByKey(lambda x, y: x + y)

## Sorting in descing order by frequency

In [0]:
named_entities_sorted = named_entities_reduced.sortBy(lambda x: -x[1])

In [0]:
named_entities_sorted.take(10)

Out[22]: [('NURSE', 91),
 ('Romeo', 71),
 ('LAWRENCE', 56),
 ('Project Gutenberg', 55),
 ('Paris', 31),
 ('Nurse', 25),
 ('PARIS', 24),
 ('thou', 21),
 ('Juliet', 21),
 ('Exit', 20)]